In [1]:
from tools.utils import random_bits
from tools.AES import *
from game.indcpa import *
from game.int_ctxt import *
from simulate import *

In [2]:
key_len = 128
block_len = 128
aes = AES(key_len, block_len)

def enc(key, m):
	if len(m) != 512:
		return None
	
	ce_0 = random_bits(128)
	cm_0 = '0'*128
	c = ce_0
	for i in range(4):
		encrypted_block =  aes.evaluate(key, xor(ce_0,m[i*block_len: (i+1)*block_len]))
		ce_0 = encrypted_block  ## updating ce_0 for ce[i-1]
		c += encrypted_block

		cm_i = aes.evaluate(key, xor(cm_0, m[i*block_len: (i+1)*block_len]))
		cm_0 = cm_i
	return (c, cm_i)

def dec(key, c):
	cipher, t = c
	if len(cipher) != 640:
		return None
	cm_0 = '0'*128
	ce_0 = cipher[:block_len]
	ce = cipher[block_len:]
	m_i = str()
	for i in range(4):
		decrypted_block = xor(aes.inverse(key, ce[i*block_len: (i+1)*block_len]), ce_0) 
		m_i += decrypted_block
		ce_0 = ce[i*block_len: (i+1)*block_len]

		cm_i = aes.evaluate(key, xor(cm_0, decrypted_block))
		cm_0 = cm_i
	if cm_i == t:
		return m_i
	else:
		return None
	


In [4]:
### Is SE IND-CPA secure?

game_indcpa = INDCPA(enc, dec, 128, 128, 10)
game_indcpa._initialize()

def adv(game):
	x = random_bits(512)
	y = random_bits(512)

	q1 = game.LR(x, x)
	c1, t1 = q1

	q2 = game.LR(x, y)
	c2, t2 = q2

	if t1 == t2:
		return 1
	else: 
		return 0
	
game_indcpa.finalize(adv(game_indcpa))



False

In [5]:
s = Simulate(game_indcpa, adv)
s.simulate_INDCPA(verbose=True)

Pr[Left => 1] = 498 / 498 = 1.0

Pr[Right => 1] = 0 / 502 = 0.0

Adv(A) = -1.0


In [6]:
## Is SE INT-CTXT secure?

game_intctxt = INTCTXT(enc, dec, 128, 20)
game_intctxt._initialize()

def adv_intctxt(game: INTCTXT):
	x = '0'*512
	cipher, tag = game.Enc(x)
	c_0 = '0'*128
	c_1 = cipher[1*128:2*128]
	c_2 = cipher[2*128: 3*128]
	c_3 = cipher[3*128 : 4*128]
	c_4 = cipher[4*128: 5*128]
	c_prime = c_0 + c_1 + c_2 + c_3 + c_4
	t_prime = c_4

	return (c_prime, t_prime)

game_intctxt.finalize(adv_intctxt(game_intctxt))

True

In [7]:
s = Simulate(game_intctxt, adv_intctxt)
s.simulate_intctxt(verbose= True)

Pr[INTCTXT_AE => 1] = 1000 / 1000 = 1.0

Adv(A) = 1.0
